# Cohesive energy scans
This Notebook follows the workflow for setting up, running, and processing the E_vs_r_scan calculations.

**Quick Notes:**

- All input scripts take key-value pairs where the "key"s correspond to the calculation's input parameters.

- Multiple calculations are prepared by specifying multiple values for the same key (on separate lines).

- The special "buildcombos" key accesses predefined functions for generating lists of input parameter values for certain sets of keys.  See documentation for more details.

__Global workflow details:__

This Notebook should be executed prior to "3. Crystal relaxation" to generate initial guesses for stable crystal structures.

**Library imports**

In [1]:
# Standard Python libraries
from __future__ import (absolute_import, print_function,
                        division, unicode_literals)
import os
from copy import deepcopy
from datetime import date
from math import floor

# http://www.numpy.org/
import numpy as np

# https://pandas.pydata.org/
import pandas as pd

# https://bokeh.pydata.org/
import bokeh
import bokeh.plotting
import bokeh.resources
import bokeh.io
import bokeh.models

from DataModelDict import DataModelDict as DM

# https://github.com/usnistgov/atomman
import atomman.unitconvert as uc

# https://github.com/usnistgov/iprPy
import iprPy
print('iprPy version', iprPy.__version__)

iprPy version 0.8.3


## 0. Access database 

### Load database

In [2]:
database = iprPy.load_database('potential_testing')

## 1. Run E_vs_r_scan calculations

This calculation performs a cohesive energy versus interatomic spacing scan using ideal atomic positions and b/a and c/a ratios to identify possible low energy crystal structures.

In [3]:
calculation = iprPy.load_calculation('E_vs_r_scan')
run_directory = iprPy.load_run_directory('potential_testing_1')

### Show calculation's allowed keys

These are the keys that the input script will accept.

In [4]:
print(calculation.allkeys)

['lammps_command', 'mpi_command', 'length_unit', 'pressure_unit', 'energy_unit', 'force_unit', 'potential_file', 'potential_content', 'potential_dir', 'load_file', 'load_content', 'load_style', 'family', 'load_options', 'symbols', 'box_parameters', 'a_uvw', 'b_uvw', 'c_uvw', 'atomshift', 'sizemults', 'minimum_r', 'maximum_r', 'number_of_steps_r']


### Write input script

Script for other pair_styles

In [13]:
input_script = """
# Commands and executables
lammps_command              lmp_mpi
mpi_command                 

# Build load information based on prototype records
buildcombos                 crystalprototype load_file prototype

# Specify prototype buildcombos limiters (only build for potential listed)
prototype_potential_record  potential_users_LAMMPS
prototype_potential_currentIPR  false

# System manipulations
a_uvw                      
b_uvw                      
c_uvw                  
atomshift                   
sizemults                   10 10 10

# Units that input/output values are in
length_unit                 
pressure_unit               
energy_unit                 
force_unit                  

# Run parameters
minimum_r                   0.5
maximum_r                   6.0
number_of_steps_r           276  
"""
with open('input_script.in', 'w') as f:
    f.write(input_script)

### Prepare calculations

In [14]:
with open('input_script.in') as f:
    input_dict = iprPy.input.parse(f, singularkeys=calculation.singularkeys)
    
database.prepare(run_directory, calculation, **input_dict)

In [15]:
database.check_records(calculation.record_style)

In database style local at C:\Users\lmh1\Documents\calculations\ipr\potential_testing :
- 10 of style calculation_E_vs_r_scan
 - 0 are complete
 - 10 still to run
 - 0 issued errors


### Run calculations

In [16]:
database.runner(run_directory)

Runner started with pid 11580
No simulations left to run


In [17]:
results_df = database.get_records_df(style=calculation.record_style)
error_df = results_df[results_df.status=='error']
print(len(error_df), 'calculations issued errors:')
for error in np.unique(error_df.error):
    print(error)

0 calculations issued errors:


## 2. Calculation analysis

In [18]:
# Settings
#outputpath = 'C:/Users/lmh1/Documents/website/calc_content'
outputpath = 'C:/Users/lmh1/Documents/potential_testing_results'
resources = bokeh.resources.Resources(mode='cdn')

### Retrieve finished calculation results

In [24]:
records = database.get_records_df(style='calculation_E_vs_r_scan', full=True, flat=False, status='finished')
pot_records = database.get_records_df(style='potential_users_LAMMPS')

### Identify compositions

In [25]:
iprPy.analysis.assign_composition(records, database)

### Define table generation function

In [26]:
def EvsRtable(df, potential, implementation, composition):
    table = '\n'.join(['# Cohesive energy (eV) vs. nearest neighbor radial distance (Angstrom).',
                       '# potential = ' + potential,
                       '# implementation = ' + implementation,
                       '# composition = ' + composition,
                       '# NOTE: These values are for static, unrelaxed structures and use the ideal',
                       '# b/a and c/a ratios for the crystal structure, not the potential-specific',
                       '# values.',
                       '',
                       '# Calculations from the NIST Interatomic Potential Repository Project:',
                       '# http://www.ctcms.nist.gov/potentials/',
                       '',
                       '# Table generated ' + str(date.today())])
    
    table += '\n\nr    '
    for series in df.itertuples():
        family = series.family
        if len(family) > 16: 
            family = family[:16]
        table += '%-16s ' %family
    table += '\n'
    
    rvalues = df.iloc[0].e_vs_r_plot.r.values
    for i in range(len(rvalues)):
        table += '%-4.2f ' % rvalues[i]
        for series in df.itertuples():
            table += '% -16.9e ' % series.e_vs_r_plot.E_coh.values[i]
        table += '\n'
    
    return table              

### Define plot generation function

In [27]:
def get_lineformats():
    lineformats = []

    lineformats.append({'family':'A1--Cu--fcc',                'color':'black',   'line':'solid'})
    lineformats.append({'family':'A2--W--bcc',                 'color':'blue',    'line':'solid'})
    lineformats.append({'family':'A3--Mg--hcp',                'color':'red',     'line':'dashed'})
    lineformats.append({'family':'A3\'--alpha-La--double-hcp', 'color':'cyan',    'line':'dashdot'})
    lineformats.append({'family':'A4--C--dc',                  'color':'magenta', 'line':'solid'})
    lineformats.append({'family':'A5--beta-Sn',                'color':'#EAC117', 'line':'solid'})
    lineformats.append({'family':'A6--In--bct',                'color':'orange',  'line':'solid'})
    lineformats.append({'family':'A7--alpha-As',               'color':'gray',    'line':'solid'})
    lineformats.append({'family':'A15--beta-W',                'color':'green',   'line':'solid'})
    lineformats.append({'family':'Ah--alpha-Po--sc',           'color':'brown',   'line':'solid'})

    lineformats.append({'family':'B1--NaCl--rock-salt',        'color':'black',   'line':'solid'})
    lineformats.append({'family':'B2--CsCl',                   'color':'blue',    'line':'solid'})
    lineformats.append({'family':'B3--ZnS--cubic-zinc-blende', 'color':'red',     'line':'solid'})
    lineformats.append({'family':'L1_0--AuCu',                 'color':'cyan',    'line':'solid'})

    lineformats.append({'family':'C1--CaF2--fluorite',         'color':'black',   'line':'solid'})

    lineformats.append({'family':'A15--Cr3Si',                 'color':'black',   'line':'solid'})
    lineformats.append({'family':'D0_3--BiF3',                 'color':'blue',    'line':'solid'})
    lineformats.append({'family':'L1_2--AuCu3',                'color':'red',     'line':'solid'})

    lineformats.append({'family':'L2_1--AlCu2Mn--heusler',     'color':'black',   'line':'solid'})
    return pd.DataFrame(lineformats)    

def EvsRplot(df, potential, implementation, composition):
    """Generate a Bokeh plot from the data"""
    
    lineformats = get_lineformats()
    
    # Initialize plot
    title = 'Cohesive Energy vs. Interatomic Spacing for %s Using %s' %(composition, implementation)
    p = bokeh.plotting.figure(title = title,
                              plot_width = 800,
                              plot_height = 600,
                              x_range = [1, 6],
                              y_range = [-10, 1],              
                              x_axis_label='r (Angstrom)', 
                              y_axis_label='Cohesive Energy (eV/atom)')
    
    # Get r values
    rvalues = df.iloc[0].e_vs_r_plot.r.values    
    
    # Loop over cohesive energies
    lowylim = -1
    for series in df.itertuples():
        family = series.family
        Evalues = deepcopy(series.e_vs_r_plot.E_coh.values)
        Evalues[np.abs(np.nan_to_num(Evalues)) > 1e5] = np.nan
        
        lineformat = lineformats[lineformats.family==family].iloc[0]
        
        # Adjust lowylim if needed
        if not np.all(np.isnan(Evalues)):
            lowy = floor(np.nanmin(Evalues))
            if lowy < lowylim:
                lowylim = lowy
        
            # Define plot line
            l = p.line(rvalues, Evalues, legend=family, 
                       line_color=lineformat.color, line_dash=lineformat.line, line_width = 2)  

            p.add_tools(bokeh.models.HoverTool(renderers=[l],
                                               tooltips=[("prototype", family),
                                                         ("r (Angstrom)", "$x"),
                                                         ("E_coh (eV)", "$y")]))
    
    if lowylim > -10:
        p.y_range = bokeh.models.Range1d(lowylim, 1)
    
    p.legend.location = "bottom_right"    
    return p  

### Create tables and plots

In [28]:
runall = True
for implememtation_key in np.unique(records.potential_LAMMPS_key):
    imp_records = records[records.potential_LAMMPS_key == implememtation_key]
    pot_record = pot_records[pot_records.key == implememtation_key].iloc[0]
    implementation = pot_record.id
    potential = pot_record.pot_id
    contentpath = os.path.join(outputpath, potential, implementation)
    
    print(implementation, end=' ')
    
    if not os.path.isdir(contentpath):
        os.makedirs(contentpath)
        isnew = True
    else:
        isnew = False
    
    if runall is True or isnew is True:
        for composition in np.unique(imp_records.composition):
            comp_records = imp_records[imp_records.composition == composition].sort_values('family')
            fstem = 'EvsR.' + composition

            table = EvsRtable(comp_records, potential, implementation, composition)
            with open(os.path.join(contentpath, fstem + '.txt'), 'w') as f:
                f.write(table)

            # Build EvsR plot
            plot = EvsRplot(comp_records, potential, implementation, composition)
            bokeh.io.save(plot, os.path.join(contentpath, fstem + '.html'),
                          resources=resources, title='Interatomic Potentials Repository Project')
            bokeh.io.export_png(plot, os.path.join(contentpath, fstem + '.png'))
            bokeh.io.reset_output()
            
    print('Done')

1999--Mishin-Y--Al--LAMMPS--testing1 Done


### Add info to PotentialProperties records

This is for generating XML records that the Interatomic Potential Repository uses to automatically build webcontent (done elsewhere).

In [29]:
# Add calculation data to PotentialProperties records
for implememtation_key in np.unique(records.potential_LAMMPS_key):
    imp_records = records[records.potential_LAMMPS_key == implememtation_key].sort_values('family')
    pot_record = pot_records[pot_records.key == implememtation_key].iloc[0]
    implementation_id = pot_record.id
    potential_id = pot_record.pot_id
    potential_key = pot_record.pot_key
    
    record_name = 'properties.' + implementation_id
    try:
        record = database.get_record(name=record_name, style='PotentialProperties')
    except:
        new = True
        content = DM()
        content['per-potential-properties'] = DM()
        content['per-potential-properties']['potential'] = DM()
        content['per-potential-properties']['potential']['key'] = potential_key
        content['per-potential-properties']['potential']['id'] = potential_id
        content['per-potential-properties']['implementation'] = DM()
        content['per-potential-properties']['implementation']['key'] = implememtation_key
        content['per-potential-properties']['implementation']['id'] = implementation_id
    else:
        content = DM(record.content)
        new = False
    
    content['per-potential-properties']['cohesive-energy-scan'] = DM()
    for composition in np.unique(imp_records.composition):
        content['per-potential-properties']['cohesive-energy-scan'].append('composition', composition)
    
    if new:
        database.add_record(name=record_name, style='PotentialProperties', content=content.xml())
    else:
        database.update_record(record=record, content=content.xml())